In [1]:
## Math/ML imports
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
#import math

## Metrics
#from sklearn.metrics import f1_score, precision_score, recall_score
#from sklearn.metrics import confusion_matrix

## Garbage collector
import gc

## Helpers
from png_helper import load_data

## Time
#import time
#from datetime import timedelta

## Graphical
#%matplotlib inline
#import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix


In [2]:
# Load data
(x_train, y_train), x_test = load_data()

# Map from 0-255 to 0-1
x_train = x_train / 255.0
y_train = y_train / 255.0
x_test = x_test / 255.0

# Turn the train set into binary (condition: >=0.5)
y_train = tf.keras.utils.to_categorical(y_train * 2) # two categorical collumns
y_train = y_train[:,:,:,1:2] # keep only second collumn. (road=1, background=0)

In [3]:
x_train.shape, x_test.shape, y_train.shape

((100, 400, 400, 3), (50, 608, 608, 3), (100, 400, 400, 1))

In [4]:
y_train[0,10,185:195,0] # road=1, background=0

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)

In [5]:
#from showim import showim
#showim(x_train[0], x_train[1], x_train[2], y_train[0], y_train[1], y_train[2])

In [6]:
x_train[0][10][200]

array([0.24705882, 0.23137255, 0.21568627])

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(4, (9, 9),
                         activation=tf.nn.relu,
                         input_shape=(400,400,3)),
  tf.keras.layers.Conv2D(8, (9, 9),
                         activation=tf.nn.relu),
  tf.keras.layers.Conv2D(12, (5, 5),
                         activation=tf.nn.relu),
  tf.keras.layers.Conv2DTranspose(1, (21, 21),
                         activation=tf.nn.sigmoid)
])

In [8]:
def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return numerator / (denominator + K.epsilon())

def bcor(y_true, y_pred):
    pp = K.mean(K.round(K.clip(y_pred, 0, 1)))
    pn = 1 - pp
    pos = K.mean(K.round(K.clip(y_true, 0, 1)))
    neg = 1 - pos
    
    tp = K.mean(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = pp - tp
    
    fn = pos - tp
    tn = pn - fn
    
    return (tp - (p*t)) / (t - (t*t))

def precision(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    return precision

def recall(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    return recall

def f1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    
    return 2*((precision * recall) / (precision + recall + K.epsilon()))

In [9]:
gc.collect()

0

In [10]:
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1, precision, recall, mcor, bcor])

In [11]:
# Train
model.fit(x_train, y_train, epochs=8)

Epoch 1/8
100/100 [==============================] - 32s 324ms/step - loss: 21.1567 - acc: 0.7232 - f1: 0.0914 - precision: 0.1726 - recall: 0.1303 - mcor: 0.0015 - bcor: 0.0038
Epoch 2/8
100/100 [==============================] - 9s 85ms/step - loss: 16.8746 - acc: 0.7995 - f1: 1.8560e-06 - precision: 0.0417 - recall: 9.2804e-07 - mcor: -4.1977e-04 - bcor: -1.3413e-06
Epoch 3/8
100/100 [==============================] - 8s 85ms/step - loss: 16.5790 - acc: 0.7995 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - mcor: -6.7757e-05 - bcor: -7.6847e-08
Epoch 4/8
100/100 [==============================] - 9s 85ms/step - loss: 16.6605 - acc: 0.7995 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - mcor: 0.0000e+00 - bcor: 0.0000e+00
Epoch 5/8
100/100 [==============================] - 8s 85ms/step - loss: 15.4840 - acc: 0.7995 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - mcor: 0.0000e+00 - bcor: 0.0000e+00
Epoch 6/8
100/100 [===================

In [12]:
# Evaluate
model.evaluate(x_train, y_train)

100/100 [==============================] - 4s 45ms/step


[15.128132963180542, 0.799531626701355, 0.0, 0.0, 0.0, 0.0, 0.0]

# Some blank lines
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>